![title](img/UNIBET.png)

In [2]:
#pip install selenium
#pip install webdriver-manager
#pip install BeautifulSoup4
#!pip install --upgrade pandas
import pandas as pd
import numpy as np
import re
import requests
import webbrowser
import seaborn as sns
import urllib.request, json 
import unicodedata
import requests
import math
import csv
import os

from unidecode import unidecode
from IPython.core.display import display, HTML
from itertools import chain
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
#from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium import webdriver
from datetime import datetime, date, timedelta
from tqdm import tqdm

pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows", None)
# driver = webdriver.Chrome(ChromeDriverManager().install())

#Function Soup
def get_page(special_match):
    driver.get(special_match)
    try:
        element = driver.find_element_by_class_name('banner-actions-container')
        element.click()
    except:
        do = "Nothing"

def strip_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFD', text)
                   if unicodedata.category(c) != 'Mn')
%run ./MatchTeamNames.ipynb

C:\Users\Valentin\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [3]:

url = "https://www.unibet.fr/zones/v3/sportnode/markets.json?nodeId=2243762&filter=R%C3%A9sultat&marketname=R%C3%A9sultat%20du%20match"
response = requests.get(url)
json_data = response.json()

#Missing:
# (dfGames["competitionName"].str.contains("Nor")) |
# (dfGames["competitionName"].str.contains("Suède, Allsvenskan")) |

row_data1 = [
    {
        "Day": day,
        "Match": e["eventName"],
        "EventId": e["eventId"],
        "competitionName": e["competitionName"],
    }
    for day, d in enumerate(json_data["marketsByType"][0]["days"])
    for e in d["events"]
]

competitions = [
#                 "Allemagne, Bundesliga",
#                 "Italie, Serie ",
#                 "France, Ligue",
#                 "Espagne, LaLiga",
                "Angleterre, Championship",
                "Angleterre, Premier League",
#                 "Portugal, Primeira Liga",
#                 "Ecosse, Premiership",
#                 "Danemark, SuperLiga",
#                 "Autriche, Bundesliga",
#                 "Suisse, Super League",
#                 "Belgique, Pro League",
#                 "Pays-Bas, Eredivisie",
#                 "Nor",
#                 "Suède, Allsvenskan",
#                 "Turquie, Süper Lig",
#                 "Grèce, Super League",
#                 "Roumanie, Liga 1",
#                 "Israël, Premier League",
#                 "Irlande, Premier League",
#                 "Irlande du Nord, NIFL Premiership",
#                 "Mexique, Liga MX",
#                 "Etats-Unis, MLS",
#                 "Argentine, Liga Profesional",
#                 "sil",
#                 "Colombie, Primera A",
#                 "Chili, Primera División",
#                 "Equateur, Liga Pro",
#                 "Paraguay, Primera Division",
#                 "Japon, J-League",
#                 "Australie, A-League",
                ]

dfGames = pd.DataFrame(row_data1)
dfGames = dfGames[(dfGames["Day"] >= -99) & (dfGames["competitionName"].isin(competitions))]

#dfGames
df = pd.DataFrame(dfGames['Match'].str.split(' - ', expand=True))
df.columns = ['Home', 'Away']
dfGames = pd.concat([df, dfGames], axis=1)
#dfGames
#dfGames.head(20)
dfGames['competitionName'].unique()

array(['France, Ligue 2 BKT®', 'Australie, A-League',
       'France, Ligue 1 UberEATS®'], dtype=object)

In [46]:
# a = pd.DataFrame(row_data1)
# a = a['competitionName'].unique()
# b = dfGames['competitionName'].unique()
# list(set(a) ^ set(b))

In [10]:
#Get match odds
import datetime
row_data0 = []
nbGames = 0

for idgame in tqdm(dfGames["EventId"]):
    g = "https://www.unibet.fr/zones/event.json?eventId=" + idgame
    with urllib.request.urlopen(g) as url:
        json_data = json.loads(url.read().decode())
        
    competitionName = json_data["eventHeader"]['competitionName']
    timestamp = datetime.datetime.fromtimestamp(json_data["eventHeader"]['eventStartDate']/1000)
    DateTime = timestamp.strftime('%Y-%m-%d %H:%M:%S')
    #home = json_data["eventHeader"]['homeName']
    #away = json_data["eventHeader"]['awayName']
    #i = -1
    if len(json_data['marketClassList'])>1:
        duel = json_data['marketClassList'][0]['marketList'][0]['selections']
#         if (" ou ") in json_data['marketClassList'][1]['marketList'][0]['selections'][1]['name']:
#             home = strip_accents(duel.split(" ou ")[0])
#             away = strip_accents(duel.split(" ou ")[1])
#         elif (" Or ") in json_data['marketClassList'][1]['marketList'][0]['selections'][1]['name']:
#             home = strip_accents(duel.split(" Or ")[0])
#             away = strip_accents(duel.split(" Or ")[1])
        home = strip_accents(duel[0]['name'])
        away = strip_accents(duel[2]['name'])

        for m in json_data['marketClassList']:
            #i = i + 1
            
            if(m['marketName'] == 'Total de buts' or
               m['marketName'] == 'Combo chance double & Total de buts'):
                for x in m['marketList']:
                    #if(m['marketName'] == 'Combo chance double & Total de buts'):
                        #print(m['marketList'][0]['selections'][0]['name'])
                    for j in x['selections']:
                        data_item = {"IdGame": idgame,
                                     "DateTime": DateTime,
                                     "Competition": competitionName,
                                     "Home": home,
                                     "Away": away,
                                     "Bet": j['name'],
                                     "Odd":round(1+float(j['currentPriceUp'])/float(j['currentPriceDown']),2)  
                                    }
                        row_data0.append(data_item)
            if(m['marketName'] == 'But pour les 2 équipes'):
                for j in m['marketList'][0]['selections']:
                    data_item = {"IdGame": idgame,
                                 "DateTime": DateTime,
                                 "Competition": competitionName,
                                 "Home": home,
                                 "Away": away,
                                 "Bet": j['name'],
                                 "Odd":round(1+float(j['currentPriceUp'])/float(j['currentPriceDown']),2)  
                                }
                    row_data0.append(data_item)
            if(m['marketName'] == 'Résultat du match' or
               m['marketName'] == 'Chance double' or
               m['marketName'] == 'Combo chance double & les 2 équipes marquent'):
                for j in m['marketList'][0]['selections']:
                    data_item = {"IdGame": idgame,
                                 "DateTime": DateTime,
                                 "Competition": competitionName,
                                 "Home": home,
                                 "Away": away,
                                 "Bet": j['name'],
                                 "Odd":round(1+float(j['currentPriceUp'])/float(j['currentPriceDown']),2)  
                                }
                    row_data0.append(data_item)

100%|██████████| 43/43 [00:10<00:00,  4.28it/s]


In [36]:
OddGames = pd.DataFrame(row_data0)
# next_sunday = datetime.date.today() + datetime.timedelta(days=(6 - datetime.date.today().weekday()) % 7 + 1)
# OddGames = OddGames[OddGames['DateTime'] < next_sunday.strftime('%Y-%m-%d 23:59:59')]
OddGames["DateTime"].unique()

Check = pd.DataFrame()
OddGames['Bet'] = OddGames['Bet'].apply(unidecode)
OddGames['Home'] = OddGames['Home'].apply(unidecode)
OddGames['Away'] = OddGames['Away'].apply(unidecode)
Check['BetOri'] = OddGames['Bet']
Check['IdGame'] = OddGames['IdGame']

OddGames["Bet"].replace(special_dict, inplace=True, regex=True)
OddGames["Away"].replace(special_dict, inplace=True, regex=True)
OddGames["Home"].replace(special_dict, inplace=True, regex=True)
inverted_dict = {v: k for k, v in Dict_teams.items()}
OddGames["Away"].replace(inverted_dict, inplace=True)
OddGames["Home"].replace(inverted_dict, inplace=True)
OddGames["Bet"].replace(inverted_dict, inplace=True)
OddGames["Bet"] = OddGames.apply(lambda p: p["Bet"].replace(p["Away"], 'A') if (p["Away"] in p["Bet"]) else p["Bet"], axis=1)
OddGames["Bet"] = OddGames.apply(lambda p: p["Bet"].replace(p["Home"], 'H') if (p["Home"] in p["Bet"]) else p["Bet"], axis=1)

OddGames["Away"].replace(Dict_teams, inplace=True)
OddGames["Home"].replace(Dict_teams, inplace=True)
OddGames["Bet"].replace(Dict_teams, inplace=True)
OddGames["Bet"] = OddGames.apply(lambda p: p["Bet"].replace(p["Away"], 'A') if (p["Away"] in p["Bet"]) else p["Bet"], axis=1)
OddGames["Bet"] = OddGames.apply(lambda p: p["Bet"].replace(p["Home"], 'H') if (p["Home"] in p["Bet"]) else p["Bet"], axis=1)

OddGames["Bet"] = OddGames["Bet"].str.replace(r'Moins de ', 'M', regex=True).replace(r'Plus de ', 'P', regex=True).replace(r' ou ', '/', regex=True)
OddGames["Bet"] = OddGames["Bet"].str.replace(r'under ', 'M', regex=True).replace(r'over ', 'P', regex=True).replace(r' Or ', '/', regex=True).replace(r' or ', '/', regex=True)
OddGames["Bet"] = OddGames["Bet"].str.replace(r'Match nul', 'D', regex=True).replace(r'match nul', 'D', regex=True).replace(r'Match Nul', 'D', regex=True)
OddGames["Bet"] = OddGames["Bet"].str.replace(r'Draw', 'D', regex=True).replace(r'Egalite', 'D', regex=True)
OddGames["Bet"] = OddGames["Bet"].replace(r'Oui', 'LDEM', regex=True).replace(r'Non', 'LDEMP', regex=True).replace(r' ', '', regex=True)
OddGames["Bet"] = OddGames["Bet"].replace(r'Yes', 'LDEM', regex=True).replace(r'No', 'LDEMP', regex=True).replace(r' ', '', regex=True)
OddGames['Bet'] = OddGames['Bet'].str.replace('.', ',', regex=False)

#OddGames.head(7)
my_list = list(set(OddGames["Bet"]))
#OddGames = OddGames.reset_index(level=1)
#OddGames = OddGames.pivot_table(values='Bet', index=OddGames.index, columns='IdGame')
OddGames2 = OddGames.set_index(['IdGame', 'DateTime', 'Home', 'Away', 'Competition']).pivot(columns='Bet').reindex()#.rename_axis(None, axis=1)

tickers_data = []
for ticker, data in OddGames2.groupby(level=0, axis=1):
    data.columns = data.columns.droplevel(0)
    data = data.reset_index()
    data.insert(0, 'ticker', ticker)
    tickers_data.append(data)

OddGames2 = pd.concat(tickers_data).drop(['ticker'], axis=1).rename_axis(None, axis=1)
#OddGames2.columns
#OddGames2.head(7)


cols = OddGames2.columns.values
Cols = [
'IdGame', 'DateTime', 'Home', 'Away', 'Competition', 
'A', 'D', 'H',  'H/D', 'D/A', 'H/A', 
    
'H/D&M1,5', 'H/D&M2,5', 'H/D&M3,5', 'H/D&M4,5',
'H/D&P1,5', 'H/D&P2,5', 'H/D&P3,5', 'H/D&P4,5', 
    
'D/A&M1,5', 'D/A&M2,5', 'D/A&M3,5', 'D/A&M4,5',
'D/A&P1,5', 'D/A&P2,5', 'D/A&P3,5', 'D/A&P4,5', 
    
'H/A&M1,5', 'H/A&M2,5', 'H/A&M3,5', 'H/A&M4,5',
'H/A&P1,5', 'H/A&P2,5', 'H/A&P3,5', 'H/A&P4,5', 
    
'LDEM', 'LDEMP', 
'H/D&LDEM', 'H/D&LDEMP', 
'D/A&LDEM', 'D/A&LDEMP', 
'H/A&LDEM', 'H/A&LDEMP', 
    
'M0,5', 'M1,5', 'M2,5', 'M3,5', 'M4,5', 'M5,5',
'P0,5', 'P1,5', 'P2,5', 'P3,5', 'P4,5', 'P5,5']
    
OddGames3 = OddGames2[Cols].reset_index(drop=True)
OddGames3['DateTime'] = pd.to_datetime(OddGames3['DateTime'])

WrongCol = []
for col in cols:
    if col not in OddGames3.columns.values:
        WrongCol.append(col)
        
mask = ~OddGames['Bet'].isin(Cols)
bugCols2 = OddGames[mask]['Bet'].tolist()
Teams = [OddGames['Away'].unique(), OddGames['Home'].unique()]
bugCols = Check[mask]['BetOri'].tolist()
# print(len(bugCols2))
print(Check[mask]['IdGame'].tolist())
# print(len(bugCols))
print(bugCols2)

Unibet = OddGames3.sort_values(by='DateTime')

2
['2657127_1', '2657126_1']
2
[array(['CD Atletico Huila', 'AD Pasto', "CD O'Higgins", 'Audax Italiano',
       'Orense SC', 'Sportivo Ameliano', 'Deportes Union La Calera',
       'Club General Caballero JLM', 'Huachipato',
       'CD Universidad Catolica', 'Once Caldas', 'Cerro Porteno',
       'Santa Fe', 'Aguilas Doradas Rionegro', 'Resistencia FC',
       'Libertad Asuncion'], dtype=object), array(['Atletico Nacional', 'Boyaca Chico', 'Deportes Copiapo',
       'Everton Vina del Mar', 'Delfin SC', 'Tacuary Asuncion',
       'Universidad de Chile', 'Coquimbo Unido', 'Guairena', 'Palestino',
       'Nublense', 'Atletico Bucaramanga', 'Club Olimpia', 'Envigado FC',
       'Independiente Medellin', 'Club Sportivo Luqueno',
       'Sportivo Trinidense'], dtype=object)]
['UnionLaCalera', 'UniversidadCatolica']


In [3211]:
# Save UNIBET odds
if Unibet.isna().mean()[Unibet.isna().mean() != 0].mean() < 0.01:
    #Save odds as CSV
    csv = pd.read_csv("Odds_Match.csv")
    def bool(p):
        if any(csv['IdGame'].str.contains(str(p))):
            return(False)
        else:
            return(True)

    new = Unibet.apply(lambda p : bool(p["IdGame"]), axis=1)
    csv = pd.concat([csv,Unibet[new]])
    csv['DateTime'] = pd.to_datetime(csv['DateTime'])
    csv.sort_values(by='DateTime').to_csv('Odds_Match.csv', index=False, encoding='utf-8')